In [18]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix_three/dw_matrix_road_sign/"

/content/drive/My Drive/Colab Notebooks/dw_matrix_three/dw_matrix_road_sign


In [4]:
ls

data/       Day2.ipynb  Day4.ipynb  logs/
Day1.ipynb  Day3.ipynb  LICENSE     README.md


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)

if y_test.ndim == 1:
  y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)

  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [11]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', ),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', ),
    MaxPool2D(),
    Dropout(0.3),


    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),


    Dense(num_classes, activation='softmax')
  ])

train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.4795 - accuracy: 0.3178 - val_loss: 0.9394 - val_accuracy: 0.6959
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.7810 - accuracy: 0.7558 - val_loss: 0.1372 - val_accuracy: 0.9588
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3543 - accuracy: 0.8920 - val_loss: 0.0720 - val_accuracy: 0.9784
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2281 - accuracy: 0.9321 - val_loss: 0.0395 - val_accuracy: 0.9890
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1734 - accuracy: 0.9483 - val_loss: 0.0291 - val_accuracy: 0.9919


0.9723356009070295

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', ),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', ),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),


    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),


    Dense(num_classes, activation='softmax')
  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]

  print(params, 'accuracy={}'.format(accuracy))

  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model }

In [34]:
space = {
    'batch_size' : hp.quniform('batch_size', 50, 200, 20)
}

best = fmin(func_obj, 
            space,
            tpe.suggest,
            1,
            Trials()
)

  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]


KeyError: ignored

In [42]:
space = {
    'batch_size' : hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0, 1),
    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0, 1)
}

best = fmin(func_obj, 
            space,
            tpe.suggest,
            30,
            Trials()
)

{'batch_size': 80.0, 'dropout_cnn_block_one': 0.6522490034834286, 'dropout_cnn_block_three': 0.9794557250380986, 'dropout_cnn_block_two': 0.9582613466226703, 'dropout_dense_block_one': 0.28415726443271094, 'dropout_dense_block_two': 0.7728321758707636}
accuracy=0.054421767592430115
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.8612928030815542, 'dropout_cnn_block_three': 0.6784101073471714, 'dropout_cnn_block_two': 0.9758004602034767, 'dropout_dense_block_one': 0.47328827980348254, 'dropout_dense_block_two': 0.40724005120075357}
accuracy=0.0476190485060215
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.6503232861180772, 'dropout_cnn_block_three': 0.05042341680386386, 'dropout_cnn_block_two': 0.4066490957189335, 'dropout_dense_block_one': 0.0506998906004229, 'dropout_dense_block_two': 0.5251621399456543}
accuracy=0.8335601091384888
{'batch_size': 80.0, 'dropout_cnn_block_one': 0.6216869059714812, 'dropout_cnn_block_three': 0.6768033796175086, 'dropout_cnn_block_two': 0.726853309102